In [1]:
# Dependencies
import requests
import json
from pprint import pprint 
import pandas as pd
from sqlalchemy import create_engine

#Create API URL
#API Variables used (as identified in the documentation here: https://api.census.gov/data/2017/acs/acs5/variables.html)   
#040 - state (41 - Oregon)
#860 - zip code tabulation area
#B00001_001E - Estimate!!Total - UNWEIGHTED SAMPLE COUNT OF THE POPULATION
#B19001_001E - Estimate!!Total - HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2017 INFLATION-ADJUSTED DOLLARS)

county_pop_url = "https://api.census.gov/data/2017/acs/acs5?get=NAME,GEO_ID,B00001_001E&for=county:*&in=state:41"
county_hhi_url = "https://api.census.gov/data/2017/acs/acs5?get=NAME,GEO_ID,B19013_001E&for=county:*&in=state:41"

# CENSUS DATA (API)

In [2]:
# Get county population information in JSON format from American Community Survey 5 Year Estimates (2017) census API
pop_response = requests.get(f"{county_pop_url}").json()

# Save 2017 county population data to dataframe
pop_by_county = pd.DataFrame(pop_response, columns = ['county','county_id', 'county_population', 'state_num','county_num']).set_index('county_id')

# Drop former header (now row 1)
pop_by_county = pop_by_county.drop('GEO_ID')

#Grab only columns that are needed
pop_by_county = pop_by_county[['county', 'county_population']]

# Remove state from County_Name column
pop_by_county['county']=pop_by_county['county'].str.replace(" County, Oregon", "")

# Preview population dataframe
pop_by_county.head()

,county,county_population
county_id,,
0500000US41047,Marion,23376
0500000US41029,Jackson,13963
0500000US41019,Douglas,8335
0500000US41023,Grant,1322
0500000US41031,Jefferson,2401


In [3]:
# Get county population information in JSON format from American Community Survey 5 Year Estimates (2017) census API
hhi_response = requests.get(f"{county_hhi_url}").json()

# Save 2017 county median household income data to dataframe
hhi_by_county = pd.DataFrame(hhi_response, columns = ['county','county_id', 'county_median_hhi', 'state_num','county_num']).set_index('county_id')

# Drop former header (now row 1)
hhi_by_county = hhi_by_county.drop('GEO_ID')

#Grab only columns that are needed
hhi_by_county = hhi_by_county[['county', 'county_median_hhi']]

# Preview population dataframe
hhi_by_county.head()

# Remove state from County_Name column
hhi_by_county['county']=hhi_by_county['county'].str.replace(" County, Oregon", "")

# Preview Median HHI by County Dataframe
hhi_by_county.head()

,county,county_median_hhi
county_id,,
0500000US41047,Marion,53828
0500000US41029,Jackson,48688
0500000US41019,Douglas,44023
0500000US41023,Grant,44826
0500000US41031,Jefferson,48464


In [4]:
# Merge county population data and county median household income data on County_ID
county_pop_hhi = pd.merge(pop_by_county, hhi_by_county, on='county')

#Preview new County Data dataframe
county_pop_hhi.head()

,county,county_population,county_median_hhi
0,Marion,23376,53828
1,Jackson,13963,48688
2,Douglas,8335,44023
3,Grant,1322,44826
4,Jefferson,2401,48464


# OREGON LIBRARY DATA (CSV)

In [5]:
# Extract Libary CSV to a dataframe
lib_file = "Resources/Oregon_Library_Directory.csv"
lib_df = pd.read_csv(lib_file)
lib_df.shape

(336, 28)

In [6]:
# Create a new data frame with specific columns needed
new_lib_df = lib_df[['Full Library Name', 'County', 'Type of Library']].copy()
new_lib_df.head()

# Create a filtered dataframe from specific columns
lib_transformed= new_lib_df.rename(columns={'Full Library Name': 'library_name',
                                                    'County': 'county',
                                                    'Type of Library': 'type'})

# Drop "County" from the county name
lib_transformed['county'] = lib_transformed['county'].str.replace(" County", "")

# Pull through data only for Public and Volunteer libraries
libraries = lib_transformed[~lib_transformed['type'].isin(['Academic', 'Special', 'Tribal'])]

libraries.head()

,library_name,county,type
1,Cannon Beach Library,Clatsop,Volunteer
2,Jackson County Library Services - Medford Bran...,Jackson,Public
4,Westfir City Library,Lane,Volunteer
5,Multnomah County Library - Sellwood-Moreland L...,Multnomah,Public
7,Aloha Community Library,Washington,Public


# OREGON SCHOOL REPORT CARDS DATA (CSV)

In [71]:
# Extract school report card CSV to a dataframe
school_file = "Resources/Oregon_Schools_Report_Card.csv"
school_df = pd.read_csv(school_file)

# school_df.dropna(inplace = True)

# school_df['On-Time Graduation']=school_df['On-Time Graduation'].astype(int)
# school_df['College Going']=school_df['College Going'].astype(int)

school_df.shape

(1253, 53)

In [72]:
# Creating a new data frame with specific columns needed
new_school_df = school_df[['School Name', 'School Type', 'County', 
                           'Student Enrollment', 'On-Time Graduation', 'College Going']].copy()
new_school_df.head(5)

# Creating a filtered dataframe from specific columns
school_transformed= new_school_df.rename(columns={'School Name': 'school_name',
                                                    'School Type': 'school_type',
                                                    'County': 'county',
                                                    'Student Enrollment': 'students_enrolled',
                                                    'On-Time Graduation': 'on_time_graduation',
                                                    'College Going': 'attending_college'})
school_transformed.head(10)

# Pull through data only for high schools
hs_data = school_transformed[~school_transformed['school_type'].isin(['E', 'M'])]

hs_data.head()

,school_name,school_type,county,students_enrolled,on_time_graduation,attending_college
2,Adrian High School,H,Malheur,89,89%,44%
3,Alsea Charter School,H,Benton,355,50%,50%
5,Amity High School,H,Yamhill,241,91%,56%
7,Eola Hills Charter School,H,Yamhill,40,38%,NaN
9,Arlington Community Charter School,H,Gilliam,167,67%,63%


# GROUPING AND JOINING DATA FRAMES

In [9]:
# Group libraries by county
county_libraries = libraries.groupby('county').count()

# Drop type column from dataframe
county_libraries = county_libraries[['library_name']]

# Rename counts to reflect that they are the total public & volunteer libraries in the county
county_libraries = county_libraries.rename(columns={'library_name':'county_libraries'})

county_libraries.reset_index(inplace=True)
county_libraries.head(100)

,county,county_libraries
0,Baker,7
1,Benton,4
2,Clackamas,14
3,Clatsop,4
4,Columbia,6
5,Coos,8
6,Crook,1
7,Curry,5
8,Deschutes,7
9,Douglas,11


In [81]:
# Group report cards by county
county_schools = hs_data.groupby('county').count()

# Drop type column from dataframe
# county_hs = county_schools.drop(columns=['school_type', 'students_enrolled', 'ela', 'math'])

# Rename counts to reflect that they are the total public schools in the county
county_hs = county_schools.rename(columns={'school_name':'county_schools'})

# county_libraries.reset_index(inplace=True)
county_hs.head()

,county_schools,school_type,students_enrolled,on_time_graduation,attending_college
county,,,,,
Baker,7,7,7,7,4
Benton,6,6,6,6,5
Clackamas,26,26,26,25,24
Clatsop,5,5,5,5,4
Columbia,7,7,7,7,5


In [10]:
# Merge Library and Census data

county_data = pd.merge(county_pop_hhi, county_libraries, how='left', on='county')
county_data.head(100)

,county,county_population,county_median_hhi,county_libraries
0,Marion,23376,53828,9.0
1,Jackson,13963,48688,16.0
2,Douglas,8335,44023,11.0
3,Grant,1322,44826,2.0
4,Jefferson,2401,48464,NaN
5,Polk,5716,56032,3.0
6,Clackamas,28560,72408,14.0
7,Deschutes,9156,59152,7.0
8,Linn,9350,49515,9.0
9,Tillamook,2598,45061,6.0
